In [20]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.metrics import mean_squared_error
import lightgbm
import pickle
import os
import sys
import pathlib

ROOT_PATH = pathlib.Path().resolve()
sys.path.append(str(ROOT_PATH))

from modules.time_series_CSV_reader import TimeSeriesCSVReader

In [3]:
ts = TimeSeriesCSVReader(2022)

In [4]:
df = ts.load_data()

2022-12-31 00:00:00

In [6]:
df.shape

(8760, 18)

In [7]:
X = df.drop(["雲量","視程","天気","日付"],axis=1)
y = df["雲量"]

In [8]:
# X["日付"] = pd.to_datetime(X["日付"])
target_column = X["風向"]
le = preprocessing.LabelEncoder()
le.fit(target_column)
label = le.transform(target_column)
X["風向"] = pd.Series(label).astype("category")

In [9]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3)

In [10]:
model = lightgbm.LGBMRegressor()
model.fit(X_train,y_train,eval_set=[(X_test, y_test), (X_train, y_train)],verbose=-1)

y_pred = model.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print('RMSE:',round(rmse,3))

RMSE: 1.666


c:\Users\astro\anaconda3\envs\forecast\lib\site-packages\lightgbm\sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "
c:\Users\astro\anaconda3\envs\forecast\lib\site-packages\lightgbm\basic.py:1780: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overriding the parameters from Reference Dataset.')
c:\Users\astro\anaconda3\envs\forecast\lib\site-packages\lightgbm\basic.py:1513: UserWarning: categorical_column in param dict is overridden.
  _log_warning(f'{cat_alias} in param dict is overridden.')


In [29]:
model_path = ROOT_PATH/"models"
model_name = "fitFromObs.pkl"
os.makedirs(model_path,exist_ok=True)

with open(model_path/model_name,"wb") as f:
    pickle.dump(model,f)

In [30]:
with open(model_path/model_name,"rb")  as f:
    road_model = pickle.load(f)